In [ ]:
from database_tools.io.records import locate_valid_records

signals = ['PLETH', 'ABP']
min_length = 75000  # 10 min in samples (125 Hz)
n_segments = 50

valid_segs = locate_valid_records(signals, min_length, n_segments)

In [ ]:
from database_tools.io.records import get_data_record

num = get_data_record('30/3000063/3000063', record_type='numerics')

In [ ]:
from database_tools.io.records import get_data_record
from database_tools.datastores import WaveformRecord

tmp = get_data_record('30/3000063/3000063_0016', record_type='waveforms')
rcd = WaveformRecord(tmp)

In [ ]:
rcd.PLETH.plot(mode='line', **dict(name='PLETH', showlegend=True))

In [37]:
from dataclasses import dataclass, InitVar, field
from database_tools.datastores import SignalStore
from database_tools.io.records import get_data_record

SIG_MAP = {
    'PLETH': 'ppg',
    'ABP': 'abp',
    'II': 'ecg_ii',
}

@dataclass
class SignalGroup:

    signals: InitVar[dict]
    ppg: SignalStore = field(init=False)
    abp: SignalStore = field(init=False)
    ecg_ii: SignalStore = field(init=False)
    _names: list = field(init=False)

    def __post_init__(self, signals):
        names = []
        for sig, store in signals.items():
            setattr(self, SIG_MAP[sig], store)
            names.append(SIG_MAP[sig])
        setattr(self, '_names', names)

    def info(self):
        out = ''
        for sig in self._names:
            store = getattr(self, sig)
            out += f'{sig: <7}- '
            out += f'Format {store.fmt}, Length of {store.shape[0]} samples\n'
        print(out)

rcd = get_data_record('30/3000063/3000063_0016', 'waveforms')

In [38]:
signals = {sig: SignalStore(rcd.p_signal[:, i], rcd.fmt[i]) for i, sig in enumerate(rcd.sig_name)}

In [39]:
group = SignalGroup(signals)

In [40]:
group.info()

ecg_ii -Format 80, Length of 1001537 samples
abp    -Format 80, Length of 1001537 samples
ppg    -Format 80, Length of 1001537 samples

